In [1]:
#Get dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup

#Set path to chromedriver
executable_path = {"executable_path": "/usr/local/bin/chromedriver"}


In [2]:

#Use Splinter and Beautiful soup for scraping
browser = Browser("chrome", **executable_path, headless=False)

#Website for news
mars_news_url = 'https://mars.nasa.gov/news/'

#Get HTML from browser
browser.visit(mars_news_url)
raw_html_news = browser.html

#Use Beautiful Soup to parse HTML
soup_news = BeautifulSoup(raw_html_news, 'lxml')

#Close browser
browser.quit()


In [3]:
#Get title and text for the first news item
title = soup_news.find('div', class_='content_title').get_text()
news = soup_news.find('div', class_='article_teaser_body').get_text()

print(title)
print(news)

Mars Helicopter to Fly on NASA’s Next Red Planet Rover Mission
NASA is adding a Mars helicopter to the agency’s next mission to the Red Planet, Mars 2020.


In [4]:
#Scrape for featured image
browser = Browser("chrome", **executable_path, headless=False)

#Website for featured image
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)
raw_html_image = browser.html
soup_image = BeautifulSoup(raw_html_image, 'lxml')
browser.quit()

# Eliminate unwated text using BeautifulSoup
featured_image = soup_image.find('a', class_='fancybox')
featured_image_url = "https://www.jpl.nasa.gov/" + featured_image["data-fancybox-href"]

print(featured_image_url)

https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA17838_ip.jpg


In [5]:

browser = Browser("chrome", **executable_path, headless=False)

#Scrape Twitter account
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)
raw_html_twitter = browser.html
soup_twitter = BeautifulSoup(raw_html_twitter, 'lxml')
browser.quit()

# Isolate tweet text and check to ensure that the tweet does indeed contain
# martian weather information. 
weather = soup_twitter.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").get_text()

weather

'Sol 2047 (May 10, 2018), Sunny, high 3C/37F, low -71C/-95F, pressure at 7.33 hPa, daylight 05:22-17:20'

In [6]:
 #Scrape Mars fact table using pandas
mars_facts_df = pd.read_html('https://space-facts.com/mars/')
mars_facts = str(mars_facts_df[0].to_html(index=False, header=False, border="0"))

mars_facts

'<table border="0" class="dataframe">\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

In [10]:
# Scrape the USGS Astrogeology website for Martian Hemisphere images
#Use Splinter and Beautiful soup for scraping
browser = Browser("chrome", **executable_path, headless=False)

#Scrape Astrogeology website
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)
raw_html_hemispheres = browser.html
soup_hemispheres = BeautifulSoup(raw_html_hemispheres, 'lxml')

hemispheres = soup_hemispheres.findAll('div', class_='description')
browser.quit()

hemispheres


[<div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/cerberus_enhanced"><h3>Cerberus Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 21 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Cerberus hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. This mosaic is composed of 104 Viking Orbiter images acquired…</p></div>,
 <div class="description"><a class="itemLink product-item" href="/search/map/Mars/Viking/schiaparelli_enhanced"><h3>Schiaparelli Hemisphere Enhanced</h3></a><span class="subtitle" style="float:left">image/tiff 35 MB</span><span class="pubDate" style="float:right"></span><br/><p>Mosaic of the Schiaparelli hemisphere of Mars projected into point perspective, a view similar to that which one would see from a spacecraft. The images were acquired in 1980 during early northern…</p></div>,
 <div class="description"><a 

In [11]:
#Loop through initial links to find download links.
hemisphere_links = []
for sphere in hemispheres:
    hemisphere_links.append('https://astrogeology.usgs.gov' + sphere.find('a')["href"])

hemisphere_links


['https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced',
 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced']

In [17]:
#Loop through links and download landing pages and search for full image links
import re
hemisphere_image_urls = []

for link in hemisphere_links:
    browser = Browser("chrome", **executable_path, headless=False)
    browser.visit(link)
    html_h = browser.html
    soup_h = BeautifulSoup(html_h, 'lxml')
    temp_h = soup_h.find("div", class_="content") 
    #print(temp_h)
    hemisphere_image_urls.append({
            "title" : temp_h.find('h2').get_text(),
            "img_url" : temp_h.find('a', href=re.compile('^http://astropedia.astrogeology.usgs.gov/download/Mars/Viking'))["href"]
        })
    browser.quit()

hemisphere_image_urls

[{'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif',
  'title': 'Valles Marineris Hemisphere Enhanced'}]